In [9]:
import glob
import os
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
%matplotlib inline

In [46]:
train = pd.read_csv('./data_dir/train.csv')
train.head(2)

,ID,Class
0,0,siren
1,1,street_music


In [11]:
X, sample_rate = librosa.load('./data_dir/Train/1.wav', res_type = 'kaiser_fast')

In [14]:
tonnetz = np.mean(librosa.feature.tonnetz(y = librosa.effects.harmonic(X), sr = sample_rate).T,axis=0)

In [15]:
len(tonnetz)

6

In [17]:
def extraction():
    train = pd.read_csv('./data_dir/train.csv')
    features = np.empty((0,187))

    Classes = train.Class.unique().tolist()
    y = []
    X = []; labels = []; new_X = []
    for i in train.ID:
        
        X, sample_rate = librosa.load('./data_dir/Train/%d.wav'%i, res_type = 'kaiser_fast')
        stft = np.abs(librosa.stft(X))
        
        mfccs = np.mean(librosa.feature.mfcc(y = X, sr = sample_rate, n_mfcc = 40).T, axis =0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        
        ext_features = np.hstack([mfccs, chroma, mel, contrast])
        features = np.vstack([features, ext_features])
        idx = Classes.index(train[train.ID == i]['Class'].tolist()[0])
        labels.append(idx)
        #print (i)
        #X.append(ext_features)
    return features, labels

In [18]:
X, labels = extraction()

/Users/lucky/anaconda3/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [35]:
X = [x for x in X]

In [39]:
X[0][:3]

array([-82.12358939, 139.50591598, -42.43086489])

In [47]:
df = pd.DataFrame(X)
df['labels'] = labels
df['Class'] = train.Class

In [56]:
print('Shape: ', df.shape)
print('Unique Labels in df, each values represents each class: ', df.labels.unique())
df.to_csv('data.csv', index = False)
print("Checking for any missing values: ", df.isnull().any().sum())
df.head()

Shape:  (5435, 189)
Unique Labels in df, each values represents each class:  [0 1 2 3 4 5 6 7 8 9]
Checking for any missing values:  0


,0,1,2,3,4,5,6,7,8,9,...,179,180,181,182,183,184,185,186,labels,Class
0,-82.123589,139.505916,-42.430865,24.827861,-11.620764,23.497084,-12.194590,25.897139,-9.405277,21.210429,...,2.831150e-07,23.510043,11.643552,13.098876,13.581493,15.363638,15.579931,38.242615,0,siren
1,-15.744005,124.119960,-29.428881,39.447193,-23.501912,16.550815,-21.736820,16.533573,-16.971729,4.483584,...,1.074347e-05,15.682245,12.363782,16.474374,17.756651,18.226688,16.084883,36.831401,1,street_music
2,-123.393651,15.181946,-50.093329,7.141872,-26.817033,-0.692504,-8.223076,13.512939,-11.382056,19.949352,...,3.851427e-05,13.333765,12.537974,14.122286,14.026329,16.014898,17.437523,33.842876,2,drilling
3,-213.278788,89.323589,-55.256190,12.632100,-47.775379,14.702909,19.039342,15.974402,-0.344623,-3.852785,...,1.134319e-06,13.875088,11.235358,21.378771,27.707693,21.865012,26.985645,46.478170,0,siren
4,-237.926479,135.902461,39.268440,21.240239,9.531328,13.885121,-3.994447,12.481487,-2.604627,6.070916,...,3.938324e-06,16.554962,12.984013,14.653759,16.328755,16.190469,16.025189,33.775486,3,dog_bark


Checking for class distribution

In [71]:
print(df.labels.value_counts()/len(df)*100)
df.Class.value_counts()/len(df)*100

8    12.290708
6    11.481141
0    11.168353
7    11.039558
3    11.039558
2    11.039558
1    11.039558
4    11.039558
9     5.630175
5     4.231831
Name: labels, dtype: float64


jackhammer          12.290708
engine_idling       11.481141
siren               11.168353
drilling            11.039558
children_playing    11.039558
dog_bark            11.039558
air_conditioner     11.039558
street_music        11.039558
car_horn             5.630175
gun_shot             4.231831
Name: Class, dtype: float64

In [74]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,labels
count,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,...,5.435000e+03,5.435000e+03,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000,5435.000000
mean,-228.151062,116.777735,-29.223291,21.179727,-13.320045,14.523744,-7.865185,11.639221,-9.897460,9.262494,...,1.233634e-01,4.034123e-01,18.361952,12.344112,15.177323,16.421931,17.733984,18.465642,35.108951,4.267157
std,132.743770,48.193549,32.041379,22.241209,16.142496,14.521131,13.248084,11.107939,10.270344,8.556911,...,1.939575e+00,6.648775e+00,5.823751,2.202442,2.955838,3.804559,3.595884,4.774269,4.436631,2.851627
min,-765.879788,-55.461749,-165.823179,-62.845176,-100.633790,-57.730385,-65.272733,-34.883915,-47.255900,-35.710833,...,2.997953e-11,3.611320e-12,9.860589,5.712008,10.108561,11.526399,11.916354,13.405572,13.127255,0.000000
25%,-313.380253,88.710914,-47.168562,9.580795,-23.499932,6.390663,-15.510185,5.671314,-16.254158,4.005852,...,2.491784e-07,9.095833e-08,13.959075,10.921031,13.182540,14.002597,15.281479,15.867467,32.826035,2.000000
50%,-216.085723,122.717151,-26.312748,22.978465,-13.218910,16.535363,-7.681295,12.264855,-9.686730,9.993290,...,1.753500e-06,6.446245e-07,17.102035,11.731881,14.379926,15.217259,16.729851,17.292074,34.603439,4.000000
75%,-129.363869,149.344125,-8.849117,35.539948,-2.158535,23.601826,-0.150936,18.111337,-3.269547,14.904179,...,1.282073e-05,3.586284e-06,21.150352,13.139180,16.254959,17.479691,19.022885,19.445478,36.686274,7.000000
max,201.331057,270.124222,64.996571,114.069868,44.862607,58.978978,54.564724,50.417440,26.963387,57.362925,...,3.580532e+01,1.615727e+02,43.338260,30.107924,34.849633,42.788570,39.521396,64.221404,62.462670,9.000000


### Machine Learning - Baseline modeling

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report

X = df.drop(['labels', 'Class'], axis =1)
y = df['labels']

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .30, random_state = 21, stratify = y)

In [84]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
Cs = [0.001, 0.1, 1]
for each in Cs:
    lg = LogisticRegression(penalty = 'l2', C = each)
    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_test)
    print("Accuracy score w/ C: ", each,accuracy_score(y_pred, y_test))
    print("Classification Report C: ",each)
    print(classification_report(y_test, y_pred))
    print("--------------------------------------------------------------------")
    
Cs = [0.001, 0.1, 1]
for each in Cs:
    lg = LogisticRegression(penalty = 'l1', C = each)
    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_test)
    print("Accuracy score w/ C: ", each,accuracy_score(y_pred, y_test))
    print("Classification Report C: ",each)
    print(classification_report(y_test, y_pred))
    print("--------------------------------------------------------------------")

Accuracy score w/ C:  0.001 0.7106069895769467
Classification Report C:  0.001
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       182
           1       0.63      0.57      0.60       180
           2       0.81      0.73      0.77       180
           3       0.69      0.56      0.62       180
           4       0.57      0.63      0.60       180
           5       0.77      0.64      0.70        69
           6       0.78      0.60      0.68       187
           7       0.58      0.77      0.66       180
           8       0.75      0.91      0.82       201
           9       0.93      0.67      0.78        92

   micro avg       0.71      0.71      0.71      1631
   macro avg       0.73      0.70      0.71      1631
weighted avg       0.72      0.71      0.71      1631

--------------------------------------------------------------------


/Users/lucky/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy score w/ C:  0.1 0.7504598405885959
Classification Report C:  0.1
              precision    recall  f1-score   support

           0       0.87      0.93      0.90       182
           1       0.66      0.59      0.63       180
           2       0.79      0.76      0.77       180
           3       0.73      0.59      0.65       180
           4       0.60      0.66      0.63       180
           5       0.83      0.77      0.80        69
           6       0.77      0.71      0.74       187
           7       0.66      0.81      0.73       180
           8       0.85      0.93      0.88       201
           9       0.84      0.75      0.79        92

   micro avg       0.75      0.75      0.75      1631
   macro avg       0.76      0.75      0.75      1631
weighted avg       0.75      0.75      0.75      1631

--------------------------------------------------------------------


/Users/lucky/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy score w/ C:  1 0.7590435315757205
Classification Report C:  1
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       182
           1       0.68      0.61      0.64       180
           2       0.79      0.74      0.76       180
           3       0.72      0.60      0.66       180
           4       0.62      0.66      0.64       180
           5       0.81      0.78      0.79        69
           6       0.81      0.78      0.79       187
           7       0.69      0.83      0.75       180
           8       0.85      0.93      0.89       201
           9       0.82      0.74      0.78        92

   micro avg       0.76      0.76      0.76      1631
   macro avg       0.76      0.76      0.76      1631
weighted avg       0.76      0.76      0.76      1631

--------------------------------------------------------------------
Accuracy score w/ C:  0.001 0.49540159411404044
Classification Report C:  0.001
              precisi